# Pandas
-----
 - Where to get help from? Stack Overflow!
 - Reference books: Python for Data Analyst by O'reilly, Learning the Pandas Library by Matt Harrison
 - Planet Python: https://planetpython.org/
 - Data Skeptic podcast: https://dataskeptic.com/





# The Series Data Structure

-------
 - The series is one of the core data structures in pandas. You think of it across between a list and a dictionary. 

In [3]:
import pandas as pd

pd.Series? # you can see the documentation, you can pass in some data, an index, and a name.


In [135]:
animals = ['Tiger', 'Bear', 'Moose']
pd.Series(animals)

0    Tiger
1     Bear
2    Moose
dtype: object

In [3]:
numbers = [1, 2, 3]
pd.Series(numbers)

0    1
1    2
2    3
dtype: int64

 - Underneath panda stores series values in a typed array using the Numpy library. This offers significant speed-up when processing data versus traditional python lists. 
 - Underneath, pandas does some type conversion. If we create a list of strings and we have one element, a None type, pandas inserts it as a None and uses the type object for the underlying array. 

In [4]:
animals = ['Tiger', 'Bear', None]
pd.Series(animals)

0    Tiger
1     Bear
2     None
dtype: object

 - NAN is not none and when we try the equality test, it's false. 
 - You need to use special functions to test for the presence of not a number, such as the Numpy library `isnan`. 

In [5]:
numbers = [1, 2, None]
pd.Series(numbers)

0    1.0
1    2.0
2    NaN
dtype: float64

In [6]:
import numpy as np
np.nan == None

False

In [7]:
np.nan == np.nan

False

In [8]:
np.isnan(np.nan)

True

 - A series can be created from dictionary data. If you do this, the index is automatically assigned to the keys of the dictionary that you provided and not just incrementing integers.   

In [9]:
sports = {'Archery': 'Bhutan',
          'Golf': 'Scotland',
          'Sumo': 'Japan',
          'Taekwondo': 'South Korea'}
s = pd.Series(sports)
s

Archery           Bhutan
Golf            Scotland
Sumo               Japan
Taekwondo    South Korea
dtype: object

 - Once the series has been created, we can get the index object using the index attribute. 

In [10]:
s.index

Index(['Archery', 'Golf', 'Sumo', 'Taekwondo'], dtype='object')

 - You could also separate your index creation from the data by passing in the index as a list explicitly to the series. 

In [11]:
s = pd.Series(['Tiger', 'Bear', 'Moose'], index = ['India', 'America', 'Canada'])
s

India      Tiger
America     Bear
Canada     Moose
dtype: object

 - Pandas overrides the automatic creation to favor only and all of the indices values that you provided. So it will ignore it from your dictionary, all keys, which are not in your index, and pandas will add non type or NAN values for any index value you provide, which is not in your dictionary key list. 

In [12]:
sports = {'Archery': 'Bhutan',
          'Golf': 'Scotland',
          'Sumo': 'Japan',
          'Taekwondo': 'South Korea'}
s = pd.Series(sports, index=['Golf', 'Sumo', 'Hockey'])
s

Golf      Scotland
Sumo         Japan
Hockey         NaN
dtype: object

# Quering a Series
-------

In [136]:
sports = {'Archery': 'Bhutan',
          'Golf': 'Scotland',
          'Sumo': 'Japan',
          'Taekwondo': 'South Korea'}
s = pd.Series(sports)
s

Archery           Bhutan
Golf            Scotland
Sumo               Japan
Taekwondo    South Korea
dtype: object

 - To query by numeric location, starting at zero, use the `iloc` attribute. 

In [137]:
s.iloc[3]

'South Korea'

 - To query by the index label, you can use the `loc` attribute.
 - Keep in mind that `iloc` and `loc` are not methods, they are attributes.

In [138]:
s.loc['Golf']

'Scotland'

In [139]:
s['Golf']

'Scotland'

 - So what happens if your index is a list of integers? This is a bit complicated, and Pandas can't determine automatically whether you're intending to query by index position or index label. 

In [140]:
sports = {99: 'Bhutan',
         100: 'Scotland',
         101: 'Japan',
         102: 'South Korea'}
s = pd.Series(sports)

In [141]:
s[0] # This won't call s.iloc[0] as one might expect, it generates an error instead

KeyError: 0

In [142]:
s.iloc[0]

'Bhutan'

 - Let's talk about working with the data. A common task is to want to consider all of the values inside of a series and want to do some sort of operation. 

In [143]:
s = pd.Series([100.00, 120.00, 101.00, 3.00])
s

0    100.0
1    120.0
2    101.0
3      3.0
dtype: float64

 - We could write a little routine which iterates over all of the items in the series and adds them together to get a total. This works, but it's slow.

In [144]:
total = 0
for item in s:
    total += item
print(total)

324.0


 - Pandas and the underlying NumPy libraries support a method of computation called vectorization. 
 - we just call `np.sum` and pass in an iterable item. 

In [145]:
import numpy as np

total = np.sum(s)
print(total)

324.0


 - `head` method reduces the amount of data printed out by the series to the first five elements. 

In [146]:
# This creats a big series of random numbers
s = pd.Series(np.random.randint(0,1000,10000))
s.head()

0    871
1    306
2    836
3    969
4    157
dtype: int64

In [147]:
len(s)

10000

#### `timeit`
- let's see if the second solution is actually faster than the other one? we can examin this with a magic function! :D
- Magic functions begin with a % sign. If we type this sign and then hit the Tab key, we can see a list of the available magic functions. 
- We're actually going to use what's called a cellular magic function. These start with two percentage signs.`%%timeit`
- You can give timeit the number of loops that you would like to run. By default, we'll use 1,000 loops. 

In [148]:
%%timeit -n 100
summary = 0
for item in s:
    summary += item

1.09 ms ± 44 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [149]:
%%timeit -n 100
summary = np.sum(s)

114 µs ± 17.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


 - Related feature in Pandas and NumPy is called broadcasting. With broadcasting, you can apply an operation to every value in the series, changing the series. 

In [150]:
s += 2 #adds two to each item in s using broadcasting
s.head()

0    873
1    308
2    838
3    971
4    159
dtype: int64

In [151]:
for label, value in s.iteritems():
    s.at[label] = value + 2
s.head()

0    875
1    310
2    840
3    973
4    161
dtype: int64

In [152]:
%%timeit -n 10
s = pd.Series(np.random.randint(0, 1000, 10000))
for label, value in s.iteritems():
    s.loc[label] = value + 2

751 ms ± 681 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [153]:
%%timeit -n 10
s = pd.Series(np.random.randint(0, 1000, 10000))
s += 2

670 µs ± 54.2 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


 - The .loc attribute lets you not only modify data in place, but also add new data as well. If the value you pass in as the index doesn't exist, then a new entry is added. And keep in mind, indices can have mixed types.
 - While it's important to be aware of the typing going on underneath, Pandas will automatically change the underlying NumPy types as appropriate. 

In [154]:
s = pd.Series([1, 2, 3])
s.loc['Animal'] = 'Bears'
s

0             1
1             2
2             3
Animal    Bears
dtype: object

 - There are a couple of important considerations when using append:
     1. Pandas is going to take your series and try to infer the best data types to use. 
     2. the append method doesn't actually change the underlying series. It instead returns a new series which is made up of the two appended together. 
     (This is actually a significant issue for new Pandas users who are used to objects being changed in place. So watch out for it, not just with append but with other Pandas functions as well.)
     3. we see that when we query the appended series for those who have cricket as their national sport, we don't get a single value, but a series itself. This is actually very common, and if you have a relational database background, this is very similar to every table query resulting in a return set which itself is a table. 
     

In [155]:
original_sports = pd.Series({'Archery': 'Bhutan',
                            'Golf': 'Scotland',
                            'Sumo': 'Japan',
                            'Taekwando': 'South Korea'})
cricket_loving_countries = pd.Series(['Autralia',
                                     'Barbados',
                                     'Pakistan',
                                     'England'],
                                    index=['Cricket',
                                          'Cricket',
                                          'Cricket',
                                          'Cricket'])
all_countries = original_sports.append(cricket_loving_countries)

In [156]:
original_sports

Archery           Bhutan
Golf            Scotland
Sumo               Japan
Taekwando    South Korea
dtype: object

In [157]:
cricket_loving_countries

Cricket    Autralia
Cricket    Barbados
Cricket    Pakistan
Cricket     England
dtype: object

In [158]:
all_countries

Archery           Bhutan
Golf            Scotland
Sumo               Japan
Taekwando    South Korea
Cricket         Autralia
Cricket         Barbados
Cricket         Pakistan
Cricket          England
dtype: object

# The DataFrame Data Structure
-------
 - The DataFrame data structure is the heart of the Panda's library. It's a primary object that you'll be working with in data analysis and cleaning tasks. 
 - The DataFrame is conceptually a two-dimensional series object, where there's an index and multiple columns of content, with each column having a label. 
 - In fact, the distinction between a column and a row is really only a conceptual distinction. And you can think of the DataFrame itself as simply a two-axes labeled array. 

In [159]:
import pandas as pd
purchase_1 = pd.Series({'Name': 'Chris',
                       'Item Purchased': 'Dog Food',
                       'Cost': 22.50})
purchase_2 = pd.Series({'Name': 'Kevin',
                       'Item Purchased': 'Kitty Litter',
                       'Cost': 2.50})
purchase_3 = pd.Series({'Name': 'Vinod',
                       'Item Purchased': 'Bird Seed',
                       'Cost': 5.00})
df = pd.DataFrame([purchase_1, purchase_2, purchase_3], index = ['Store 1', 'Store 1', 'Store 2'])
df.head()

Name Item Purchased  Cost
Store 1  Chris       Dog Food  22.5
Store 1  Kevin   Kitty Litter   2.5
Store 2  Vinod      Bird Seed   5.0

 - Because the DataFrame is two-dimensional, passing a single value to the `loc` indexing operator will return series if there's only one row to return. 

In [160]:
df.loc['Store 2']

Name                  Vinod
Item Purchased    Bird Seed
Cost                      5
Name: Store 2, dtype: object

In [161]:
type(df.loc['Store 2'])

pandas.core.series.Series

 - It's important to remember that the indices and column names along either axes, horizontal or vertical, could be non-unique. 
 - For instance, in this example, we see two purchase records for Store 1 as different rows. If we use a single value with the DataFrame `loc` attribute, multiple rows of the DataFrame will return, not as a new series, but as a new DataFrame. 

In [162]:
df.loc['Store 1']

Name Item Purchased  Cost
Store 1  Chris       Dog Food  22.5
Store 1  Kevin   Kitty Litter   2.5

 - One of the powers of the Panda's DataFrame is that you can quickly select data based on multiple axes. 

In [164]:
df.loc['Store 1', 'Cost']

Store 1    22.5
Store 1     2.5
Name: Cost, dtype: float64

 - What if we just wanted to do column selection and just get a list of all of the costs? 
     1. First, you can get a transpose of the DataFrame, using the capital T attribute, which swaps all of the columns and rows. This essentially turns your column names into indices. And we can then use the `loc` method. This works, but it's pretty ugly:

In [165]:
df.T

Store 1       Store 1    Store 2
Name               Chris         Kevin      Vinod
Item Purchased  Dog Food  Kitty Litter  Bird Seed
Cost                22.5           2.5          5

In [166]:
df.T.loc['Cost']

Store 1    22.5
Store 1     2.5
Store 2       5
Name: Cost, dtype: object

 - Since iloc and loc are used for row selection, the Panda's developers reserved indexing operator directly on the DataFrame for column selection. 
 - In a Panda's DataFrame, columns always have a name. So this selection is always label based
     2. So the second way to do it is by simply using indexing operator:
 

In [167]:
df['Cost']

Store 1    22.5
Store 1     2.5
Store 2     5.0
Name: Cost, dtype: float64

 - You can also chain operations together.
 - Chaining can come with some costs and is best avoided if you can use another approach. In particular, chaining tends to cause Pandas to return a copy of the DataFrame instead of a view on the DataFrame. 
 - For selecting a data, this is not a big deal, though it might be slower than necessary. If you are changing data though, this is an important distinction and can be a source of error. 

In [168]:
df.loc['Store 1']['Cost']

Store 1    22.5
Store 1     2.5
Name: Cost, dtype: float64

 - `.loc` also supports slicing. If we wanted to select all rows, we can use a column to indicate a full slice from beginning to end. And then add the column name as the second parameter as a string. In fact, if we wanted to include multiply columns, we could do so in a list. And Pandas will bring back only the columns we have asked for. 

In [170]:
df.loc[:, ['Name', 'Cost']]

Name  Cost
Store 1  Chris  22.5
Store 1  Kevin   2.5
Store 2  Vinod   5.0

 - It's easy to delete data in series and DataFrames, and we can use the `drop` function to do so. 
 - This function takes a single parameter, which is the index or roll label, to drop. 
 - The drop function doesn't change the DataFrame by default. And instead, returns to you a copy of the DataFrame with the given rows removed. 

In [174]:
df.drop('Store 1')

Name Item Purchased  Cost
Store 2  Vinod      Bird Seed   5.0

In [175]:
df

Name Item Purchased  Cost
Store 1  Chris       Dog Food  22.5
Store 1  Kevin   Kitty Litter   2.5
Store 2  Vinod      Bird Seed   5.0

 - Let's make a copy with the copy method and do a drop on it instead. This is a very typical pattern in Pandas, where in place changes to a DataFrame are only done if need be, usually on changes involving indices. 

In [176]:
copy_df = df.copy()
copy_df = copy_df.drop('Store 1')
copy_df

Name Item Purchased  Cost
Store 2  Vinod      Bird Seed   5.0

 - Drop has two interesting optional parameters. The first is called in place, and if it's set to true, the DataFrame will be updated in place, instead of a copy being returned. 
 - The second parameter is the axes, which should be dropped. By default, this value is 0, indicating the row axes. But you could change it to 1 if you want to drop a column. 

In [92]:
copy_df.drop?

In [93]:
del copy_df['Name']
copy_df

Item Purchased  Cost
Store 2   Kitty Litter   2.5
Store 3      Bird Seed   5.0

In [94]:
df['Location'] = None
df

Name Item Purchased  Cost Location
Store 1  Chris       Dog Food  22.5     None
Store 2  Kevin   Kitty Litter   2.5     None
Store 3  Vinod      Bird Seed   5.0     None

# Dataframe Indexing and Loading

In [95]:
costs = df['Cost']
costs

Store 1    22.5
Store 2     2.5
Store 3     5.0
Name: Cost, dtype: float64

In [96]:
costs += 2
costs

Store 1    24.5
Store 2     4.5
Store 3     7.0
Name: Cost, dtype: float64

In [97]:
df

Name Item Purchased  Cost Location
Store 1  Chris       Dog Food  24.5     None
Store 2  Kevin   Kitty Litter   4.5     None
Store 3  Vinod      Bird Seed   7.0     None

In [98]:
!cat olympics.csv

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
,№ Summer,01 !,02 !,03 !,Total,№ Winter,01 !,02 !,03 !,Total,№ Games,01 !,02 !,03 !,Combined total
Afghanistan (AFG),13,0,0,2,2,0,0,0,0,0,13,0,0,2,2
Algeria (ALG),12,5,2,8,15,3,0,0,0,0,15,5,2,8,15
Argentina (ARG),23,18,24,28,70,18,0,0,0,0,41,18,24,28,70
Armenia (ARM),5,1,2,9,12,6,0,0,0,0,11,1,2,9,12
Australasia (ANZ) [ANZ],2,3,4,5,12,0,0,0,0,0,2,3,4,5,12
Australia (AUS) [AUS] [Z],25,139,152,177,468,18,5,3,4,12,43,144,155,181,480
Austria (AUT),26,18,33,35,86,22,59,78,81,218,48,77,111,116,304
Azerbaijan (AZE),5,6,5,15,26,5,0,0,0,0,10,6,5,15,26
Bahamas (BAH),15,5,2,5,12,0,0,0,0,0,15,5,2,5,12
Bahrain (BRN),8,0,0,1,1,0,0,0,0,0,8,0,0,1,1
Barbados (BAR) [BAR],11,0,0,1,1,0,0,0,0,0,11,0,0,1,1
Belarus (BLR),5,12,24,39,75,6,6,4,5,15,11,18,28,44,90
Belgium (BEL),25,37,52,53,142,20,1,1,3,5,45,38,53,56,147
Bermuda (BER),17,0,0,1,1,7,0,0,0,0,24,0,0,1,1
Bohemia (BOH) [BOH] [Z],3,0,1,3,4,0,0,0,0,0,3,0,1,3,4
Botswana (BOT),9,0,1,0,1,0,0,0,0,0,9,0,1,0,1
Brazil (BRA),

In [99]:
df = pd.read_csv('olympics.csv')
df.head()

0         1     2     3     4      5         6     7     8  \
0                NaN  № Summer  01 !  02 !  03 !  Total  № Winter  01 !  02 !   
1  Afghanistan (AFG)        13     0     0     2      2         0     0     0   
2      Algeria (ALG)        12     5     2     8     15         3     0     0   
3    Argentina (ARG)        23    18    24    28     70        18     0     0   
4      Armenia (ARM)         5     1     2     9     12         6     0     0   

      9     10       11    12    13    14              15  
0  03 !  Total  № Games  01 !  02 !  03 !  Combined total  
1     0      0       13     0     0     2               2  
2     0      0       15     5     2     8              15  
3     0      0       41    18    24    28              70  
4     0      0       11     1     2     9              12

In [100]:
df = pd.read_csv('olympics.csv', index_col = 0, skiprows = 1)
df.head()

№ Summer  01 !  02 !  03 !  Total  № Winter  01 !.1  \
Afghanistan (AFG)              13     0     0     2      2         0       0   
Algeria (ALG)                  12     5     2     8     15         3       0   
Argentina (ARG)                23    18    24    28     70        18       0   
Armenia (ARM)                   5     1     2     9     12         6       0   
Australasia (ANZ) [ANZ]         2     3     4     5     12         0       0   

                         02 !.1  03 !.1  Total.1  № Games  01 !.2  02 !.2  \
Afghanistan (AFG)             0       0        0       13       0       0   
Algeria (ALG)                 0       0        0       15       5       2   
Argentina (ARG)               0       0        0       41      18      24   
Armenia (ARM)                 0       0        0       11       1       2   
Australasia (ANZ) [ANZ]       0       0        0        2       3       4   

                         03 !.2  Combined total  
Afghanistan (AFG)             2               2  
Algeria (ALG)                 8              15  
Argentina (ARG)              28              70  
Armenia (ARM)                 9              12  
Australasia (ANZ) [ANZ]       5              12

In [101]:
df.columns

Index(['№ Summer', '01 !', '02 !', '03 !', 'Total', '№ Winter', '01 !.1',
       '02 !.1', '03 !.1', 'Total.1', '№ Games', '01 !.2', '02 !.2', '03 !.2',
       'Combined total'],
      dtype='object')

In [102]:
for col in df.columns:
    if col[:2] == '01':
        df.rename(columns = {col:'Gold' + col[4:]}, inplace = True)
    if col[:2] == '02':
        df.rename(columns = {col:'Silver' + col[4:]}, inplace = True)
    if col[:2] == '03':
        df.rename(columns = {col:'Bronze' + col[4:]}, inplace = True)
    if col[:1] == '№':
        df.rename(columns = {col:'#' + col[1:]}, inplace = True)

df.head()

# Summer  Gold  Silver  Bronze  Total  # Winter  \
Afghanistan (AFG)              13     0       0       2      2         0   
Algeria (ALG)                  12     5       2       8     15         3   
Argentina (ARG)                23    18      24      28     70        18   
Armenia (ARM)                   5     1       2       9     12         6   
Australasia (ANZ) [ANZ]         2     3       4       5     12         0   

                         Gold.1  Silver.1  Bronze.1  Total.1  # Games  Gold.2  \
Afghanistan (AFG)             0         0         0        0       13       0   
Algeria (ALG)                 0         0         0        0       15       5   
Argentina (ARG)               0         0         0        0       41      18   
Armenia (ARM)                 0         0         0        0       11       1   
Australasia (ANZ) [ANZ]       0         0         0        0        2       3   

                         Silver.2  Bronze.2  Combined total  
Afghanistan (AFG)               0         2               2  
Algeria (ALG)                   2         8              15  
Argentina (ARG)                24        28              70  
Armenia (ARM)                   2         9              12  
Australasia (ANZ) [ANZ]         4         5              12

# Quering a DataFrame

In [103]:
df['Gold'] > 0

Afghanistan (AFG)                               False
Algeria (ALG)                                    True
Argentina (ARG)                                  True
Armenia (ARM)                                    True
Australasia (ANZ) [ANZ]                          True
                                                ...  
Independent Olympic Participants (IOP) [IOP]    False
Zambia (ZAM) [ZAM]                              False
Zimbabwe (ZIM) [ZIM]                             True
Mixed team (ZZX) [ZZX]                           True
Totals                                           True
Name: Gold, Length: 147, dtype: bool

In [104]:
only_gold = df.where(df['Gold'] > 0)
only_gold.head()

# Summer  Gold  Silver  Bronze  Total  # Winter  \
Afghanistan (AFG)             NaN   NaN     NaN     NaN    NaN       NaN   
Algeria (ALG)                12.0   5.0     2.0     8.0   15.0       3.0   
Argentina (ARG)              23.0  18.0    24.0    28.0   70.0      18.0   
Armenia (ARM)                 5.0   1.0     2.0     9.0   12.0       6.0   
Australasia (ANZ) [ANZ]       2.0   3.0     4.0     5.0   12.0       0.0   

                         Gold.1  Silver.1  Bronze.1  Total.1  # Games  Gold.2  \
Afghanistan (AFG)           NaN       NaN       NaN      NaN      NaN     NaN   
Algeria (ALG)               0.0       0.0       0.0      0.0     15.0     5.0   
Argentina (ARG)             0.0       0.0       0.0      0.0     41.0    18.0   
Armenia (ARM)               0.0       0.0       0.0      0.0     11.0     1.0   
Australasia (ANZ) [ANZ]     0.0       0.0       0.0      0.0      2.0     3.0   

                         Silver.2  Bronze.2  Combined total  
Afghanistan (AFG)             NaN       NaN             NaN  
Algeria (ALG)                 2.0       8.0            15.0  
Argentina (ARG)              24.0      28.0            70.0  
Armenia (ARM)                 2.0       9.0            12.0  
Australasia (ANZ) [ANZ]       4.0       5.0            12.0

In [105]:
only_gold['Gold'].count()

100

In [106]:
df['Gold'].count()

147

In [108]:
only_gold = only_gold.dropna()
only_gold.head()

# Summer   Gold  Silver  Bronze  Total  # Winter  \
Algeria (ALG)                  12.0    5.0     2.0     8.0   15.0       3.0   
Argentina (ARG)                23.0   18.0    24.0    28.0   70.0      18.0   
Armenia (ARM)                   5.0    1.0     2.0     9.0   12.0       6.0   
Australasia (ANZ) [ANZ]         2.0    3.0     4.0     5.0   12.0       0.0   
Australia (AUS) [AUS] [Z]      25.0  139.0   152.0   177.0  468.0      18.0   

                           Gold.1  Silver.1  Bronze.1  Total.1  # Games  \
Algeria (ALG)                 0.0       0.0       0.0      0.0     15.0   
Argentina (ARG)               0.0       0.0       0.0      0.0     41.0   
Armenia (ARM)                 0.0       0.0       0.0      0.0     11.0   
Australasia (ANZ) [ANZ]       0.0       0.0       0.0      0.0      2.0   
Australia (AUS) [AUS] [Z]     5.0       3.0       4.0     12.0     43.0   

                           Gold.2  Silver.2  Bronze.2  Combined total  
Algeria (ALG)                 5.0       2.0       8.0            15.0  
Argentina (ARG)              18.0      24.0      28.0            70.0  
Armenia (ARM)                 1.0       2.0       9.0            12.0  
Australasia (ANZ) [ANZ]       3.0       4.0       5.0            12.0  
Australia (AUS) [AUS] [Z]   144.0     155.0     181.0           480.0

In [109]:
only_gold = df[df['Gold'] > 0]
only_gold.head()

# Summer  Gold  Silver  Bronze  Total  # Winter  \
Algeria (ALG)                    12     5       2       8     15         3   
Argentina (ARG)                  23    18      24      28     70        18   
Armenia (ARM)                     5     1       2       9     12         6   
Australasia (ANZ) [ANZ]           2     3       4       5     12         0   
Australia (AUS) [AUS] [Z]        25   139     152     177    468        18   

                           Gold.1  Silver.1  Bronze.1  Total.1  # Games  \
Algeria (ALG)                   0         0         0        0       15   
Argentina (ARG)                 0         0         0        0       41   
Armenia (ARM)                   0         0         0        0       11   
Australasia (ANZ) [ANZ]         0         0         0        0        2   
Australia (AUS) [AUS] [Z]       5         3         4       12       43   

                           Gold.2  Silver.2  Bronze.2  Combined total  
Algeria (ALG)                   5         2         8              15  
Argentina (ARG)                18        24        28              70  
Armenia (ARM)                   1         2         9              12  
Australasia (ANZ) [ANZ]         3         4         5              12  
Australia (AUS) [AUS] [Z]     144       155       181             480

In [110]:
len(df[(df['Gold'] > 0) | (df['Gold.1'] > 0)])

101

In [111]:
df[(df['Gold.1'] > 0) & (df['Gold'] == 0)]

# Summer  Gold  Silver  Bronze  Total  # Winter  Gold.1  \
Liechtenstein (LIE)        16     0       0       0      0        18       2   

                     Silver.1  Bronze.1  Total.1  # Games  Gold.2  Silver.2  \
Liechtenstein (LIE)         2         5        9       34       2         2   

                     Bronze.2  Combined total  
Liechtenstein (LIE)         5               9

# Indexing DataFrames

In [112]:
df.head()

# Summer  Gold  Silver  Bronze  Total  # Winter  \
Afghanistan (AFG)              13     0       0       2      2         0   
Algeria (ALG)                  12     5       2       8     15         3   
Argentina (ARG)                23    18      24      28     70        18   
Armenia (ARM)                   5     1       2       9     12         6   
Australasia (ANZ) [ANZ]         2     3       4       5     12         0   

                         Gold.1  Silver.1  Bronze.1  Total.1  # Games  Gold.2  \
Afghanistan (AFG)             0         0         0        0       13       0   
Algeria (ALG)                 0         0         0        0       15       5   
Argentina (ARG)               0         0         0        0       41      18   
Armenia (ARM)                 0         0         0        0       11       1   
Australasia (ANZ) [ANZ]       0         0         0        0        2       3   

                         Silver.2  Bronze.2  Combined total  
Afghanistan (AFG)               0         2               2  
Algeria (ALG)                   2         8              15  
Argentina (ARG)                24        28              70  
Armenia (ARM)                   2         9              12  
Australasia (ANZ) [ANZ]         4         5              12

In [113]:
df['country'] = df.index
df = df.set_index('Gold')
df.head()

# Summer  Silver  Bronze  Total  # Winter  Gold.1  Silver.1  Bronze.1  \
Gold                                                                          
0           13       0       2      2         0       0         0         0   
5           12       2       8     15         3       0         0         0   
18          23      24      28     70        18       0         0         0   
1            5       2       9     12         6       0         0         0   
3            2       4       5     12         0       0         0         0   

      Total.1  # Games  Gold.2  Silver.2  Bronze.2  Combined total  \
Gold                                                                 
0           0       13       0         0         2               2   
5           0       15       5         2         8              15   
18          0       41      18        24        28              70   
1           0       11       1         2         9              12   
3           0        2       3         4         5              12   

                      country  
Gold                           
0           Afghanistan (AFG)  
5               Algeria (ALG)  
18            Argentina (ARG)  
1               Armenia (ARM)  
3     Australasia (ANZ) [ANZ]

In [114]:
df = df.reset_index()
df.head()

Gold  # Summer  Silver  Bronze  Total  # Winter  Gold.1  Silver.1  \
0     0        13       0       2      2         0       0         0   
1     5        12       2       8     15         3       0         0   
2    18        23      24      28     70        18       0         0   
3     1         5       2       9     12         6       0         0   
4     3         2       4       5     12         0       0         0   

   Bronze.1  Total.1  # Games  Gold.2  Silver.2  Bronze.2  Combined total  \
0         0        0       13       0         0         2               2   
1         0        0       15       5         2         8              15   
2         0        0       41      18        24        28              70   
3         0        0       11       1         2         9              12   
4         0        0        2       3         4         5              12   

                   country  
0        Afghanistan (AFG)  
1            Algeria (ALG)  
2          Argentina (ARG)  
3            Armenia (ARM)  
4  Australasia (ANZ) [ANZ]

In [116]:
df = pd.read_csv('census.csv')
df.head()

SUMLEV  REGION  DIVISION  STATE  COUNTY   STNAME         CTYNAME  \
0      40       3         6      1       0  Alabama         Alabama   
1      50       3         6      1       1  Alabama  Autauga County   
2      50       3         6      1       3  Alabama  Baldwin County   
3      50       3         6      1       5  Alabama  Barbour County   
4      50       3         6      1       7  Alabama     Bibb County   

   CENSUS2010POP  ESTIMATESBASE2010  POPESTIMATE2010  ...  RDOMESTICMIG2011  \
0        4779736            4780127          4785161  ...          0.002295   
1          54571              54571            54660  ...          7.242091   
2         182265             182265           183193  ...         14.832960   
3          27457              27457            27341  ...         -4.728132   
4          22915              22919            22861  ...         -5.527043   

   RDOMESTICMIG2012  RDOMESTICMIG2013  RDOMESTICMIG2014  RDOMESTICMIG2015  \
0         -0.193196          0.381066          0.582002         -0.467369   
1         -2.915927         -3.012349          2.265971         -2.530799   
2         17.647293         21.845705         19.243287         17.197872   
3         -2.500690         -7.056824         -3.904217        -10.543299   
4         -5.068871         -6.201001         -0.177537          0.177258   

   RNETMIG2011  RNETMIG2012  RNETMIG2013  RNETMIG2014  RNETMIG2015  
0     1.030015     0.826644     1.383282     1.724718     0.712594  
1     7.606016    -2.626146    -2.722002     2.592270    -2.187333  
2    15.844176    18.559627    22.727626    20.317142    18.293499  
3    -4.874741    -2.758113    -7.167664    -3.978583   -10.543299  
4    -5.088389    -4.363636    -5.403729     0.754533     1.107861  

[5 rows x 100 columns]

In [117]:
df['SUMLEV'].unique()

array([40, 50])

In [118]:
df = df[df['SUMLEV'] == 50]
df.head()

SUMLEV  REGION  DIVISION  STATE  COUNTY   STNAME         CTYNAME  \
1      50       3         6      1       1  Alabama  Autauga County   
2      50       3         6      1       3  Alabama  Baldwin County   
3      50       3         6      1       5  Alabama  Barbour County   
4      50       3         6      1       7  Alabama     Bibb County   
5      50       3         6      1       9  Alabama   Blount County   

   CENSUS2010POP  ESTIMATESBASE2010  POPESTIMATE2010  ...  RDOMESTICMIG2011  \
1          54571              54571            54660  ...          7.242091   
2         182265             182265           183193  ...         14.832960   
3          27457              27457            27341  ...         -4.728132   
4          22915              22919            22861  ...         -5.527043   
5          57322              57322            57373  ...          1.807375   

   RDOMESTICMIG2012  RDOMESTICMIG2013  RDOMESTICMIG2014  RDOMESTICMIG2015  \
1         -2.915927         -3.012349          2.265971         -2.530799   
2         17.647293         21.845705         19.243287         17.197872   
3         -2.500690         -7.056824         -3.904217        -10.543299   
4         -5.068871         -6.201001         -0.177537          0.177258   
5         -1.177622         -1.748766         -2.062535         -1.369970   

   RNETMIG2011  RNETMIG2012  RNETMIG2013  RNETMIG2014  RNETMIG2015  
1     7.606016    -2.626146    -2.722002     2.592270    -2.187333  
2    15.844176    18.559627    22.727626    20.317142    18.293499  
3    -4.874741    -2.758113    -7.167664    -3.978583   -10.543299  
4    -5.088389    -4.363636    -5.403729     0.754533     1.107861  
5     1.859511    -0.848580    -1.402476    -1.577232    -0.884411  

[5 rows x 100 columns]

In [120]:
columns_to_keep = ['STNAME',
                  'CTYNAME',
                  'BIRTHS2010',
                  'BIRTHS2011',
                  'BIRTHS2012',
                  'BIRTHS2013',
                  'BIRTHS2014',
                  'BIRTHS2015',
                  'POPESTIMATE2010',
                  'POPESTIMATE2011',
                  'POPESTIMATE2012',
                  'POPESTIMATE2013',
                  'POPESTIMATE2014',
                  'POPESTIMATE2015']
df = df[columns_to_keep]
df.head()

STNAME         CTYNAME  BIRTHS2010  BIRTHS2011  BIRTHS2012  BIRTHS2013  \
1  Alabama  Autauga County         151         636         615         574   
2  Alabama  Baldwin County         517        2187        2092        2160   
3  Alabama  Barbour County          70         335         300         283   
4  Alabama     Bibb County          44         266         245         259   
5  Alabama   Blount County         183         744         710         646   

   BIRTHS2014  BIRTHS2015  POPESTIMATE2010  POPESTIMATE2011  POPESTIMATE2012  \
1         623         600            54660            55253            55175   
2        2186        2240           183193           186659           190396   
3         260         269            27341            27226            27159   
4         247         253            22861            22733            22642   
5         618         603            57373            57711            57776   

   POPESTIMATE2013  POPESTIMATE2014  POPESTIMATE2015  
1            55038            55290            55347  
2           195126           199713           203709  
3            26973            26815            26489  
4            22512            22549            22583  
5            57734            57658            57673

In [121]:
df = df.set_index(['STNAME', 'CTYNAME'])
df.head()

BIRTHS2010  BIRTHS2011  BIRTHS2012  BIRTHS2013  \
STNAME  CTYNAME                                                          
Alabama Autauga County         151         636         615         574   
        Baldwin County         517        2187        2092        2160   
        Barbour County          70         335         300         283   
        Bibb County             44         266         245         259   
        Blount County          183         744         710         646   

                        BIRTHS2014  BIRTHS2015  POPESTIMATE2010  \
STNAME  CTYNAME                                                   
Alabama Autauga County         623         600            54660   
        Baldwin County        2186        2240           183193   
        Barbour County         260         269            27341   
        Bibb County            247         253            22861   
        Blount County          618         603            57373   

                        POPESTIMATE2011  POPESTIMATE2012  POPESTIMATE2013  \
STNAME  CTYNAME                                                             
Alabama Autauga County            55253            55175            55038   
        Baldwin County           186659           190396           195126   
        Barbour County            27226            27159            26973   
        Bibb County               22733            22642            22512   
        Blount County             57711            57776            57734   

                        POPESTIMATE2014  POPESTIMATE2015  
STNAME  CTYNAME                                           
Alabama Autauga County            55290            55347  
        Baldwin County           199713           203709  
        Barbour County            26815            26489  
        Bibb County               22549            22583  
        Blount County             57658            57673

In [122]:
df.loc['Michigan', 'Washtenaw County']

BIRTHS2010            977
BIRTHS2011           3826
BIRTHS2012           3780
BIRTHS2013           3662
BIRTHS2014           3683
BIRTHS2015           3709
POPESTIMATE2010    345563
POPESTIMATE2011    349048
POPESTIMATE2012    351213
POPESTIMATE2013    354289
POPESTIMATE2014    357029
POPESTIMATE2015    358880
Name: (Michigan, Washtenaw County), dtype: int64

In [123]:
df.loc[[('Michigan', 'Washtenaw County'),
        ('Michigan', 'Wayne County')]]

BIRTHS2010  BIRTHS2011  BIRTHS2012  BIRTHS2013  \
STNAME   CTYNAME                                                            
Michigan Washtenaw County         977        3826        3780        3662   
         Wayne County            5918       23819       23270       23377   

                           BIRTHS2014  BIRTHS2015  POPESTIMATE2010  \
STNAME   CTYNAME                                                     
Michigan Washtenaw County        3683        3709           345563   
         Wayne County           23607       23586          1815199   

                           POPESTIMATE2011  POPESTIMATE2012  POPESTIMATE2013  \
STNAME   CTYNAME                                                               
Michigan Washtenaw County           349048           351213           354289   
         Wayne County              1801273          1792514          1775713   

                           POPESTIMATE2014  POPESTIMATE2015  
STNAME   CTYNAME                                             
Michigan Washtenaw County           357029           358880  
         Wayne County              1766008          1759335

# Missing values

In [124]:
df = pd.read_csv('log.csv')
df

time    user          video  playback position paused  volume
0   1469974424  cheryl     intro.html                  5  False    10.0
1   1469974454  cheryl     intro.html                  6    NaN     NaN
2   1469974544  cheryl     intro.html                  9    NaN     NaN
3   1469974574  cheryl     intro.html                 10    NaN     NaN
4   1469977514     bob     intro.html                  1    NaN     NaN
5   1469977544     bob     intro.html                  1    NaN     NaN
6   1469977574     bob     intro.html                  1    NaN     NaN
7   1469977604     bob     intro.html                  1    NaN     NaN
8   1469974604  cheryl     intro.html                 11    NaN     NaN
9   1469974694  cheryl     intro.html                 14    NaN     NaN
10  1469974724  cheryl     intro.html                 15    NaN     NaN
11  1469974454     sue  advanced.html                 24    NaN     NaN
12  1469974524     sue  advanced.html                 25    NaN     NaN
13  1469974424     sue  advanced.html                 23  False    10.0
14  1469974554     sue  advanced.html                 26    NaN     NaN
15  1469974624     sue  advanced.html                 27    NaN     NaN
16  1469974654     sue  advanced.html                 28    NaN     5.0
17  1469974724     sue  advanced.html                 29    NaN     NaN
18  1469974484  cheryl     intro.html                  7    NaN     NaN
19  1469974514  cheryl     intro.html                  8    NaN     NaN
20  1469974754     sue  advanced.html                 30    NaN     NaN
21  1469974824     sue  advanced.html                 31    NaN     NaN
22  1469974854     sue  advanced.html                 32    NaN     NaN
23  1469974924     sue  advanced.html                 33    NaN     NaN
24  1469977424     bob     intro.html                  1   True    10.0
25  1469977454     bob     intro.html                  1    NaN     NaN
26  1469977484     bob     intro.html                  1    NaN     NaN
27  1469977634     bob     intro.html                  1    NaN     NaN
28  1469977664     bob     intro.html                  1    NaN     NaN
29  1469974634  cheryl     intro.html                 12    NaN     NaN
30  1469974664  cheryl     intro.html                 13    NaN     NaN
31  1469977694     bob     intro.html                  1    NaN     NaN
32  1469977724     bob     intro.html                  1    NaN     NaN

In [125]:
df.fillna?

In [127]:
df = df.set_index('time')
df = df.sort_index()
df

user          video  playback position paused  volume
time                                                               
1469974424  cheryl     intro.html                  5  False    10.0
1469974424     sue  advanced.html                 23  False    10.0
1469974454  cheryl     intro.html                  6    NaN     NaN
1469974454     sue  advanced.html                 24    NaN     NaN
1469974484  cheryl     intro.html                  7    NaN     NaN
1469974514  cheryl     intro.html                  8    NaN     NaN
1469974524     sue  advanced.html                 25    NaN     NaN
1469974544  cheryl     intro.html                  9    NaN     NaN
1469974554     sue  advanced.html                 26    NaN     NaN
1469974574  cheryl     intro.html                 10    NaN     NaN
1469974604  cheryl     intro.html                 11    NaN     NaN
1469974624     sue  advanced.html                 27    NaN     NaN
1469974634  cheryl     intro.html                 12    NaN     NaN
1469974654     sue  advanced.html                 28    NaN     5.0
1469974664  cheryl     intro.html                 13    NaN     NaN
1469974694  cheryl     intro.html                 14    NaN     NaN
1469974724  cheryl     intro.html                 15    NaN     NaN
1469974724     sue  advanced.html                 29    NaN     NaN
1469974754     sue  advanced.html                 30    NaN     NaN
1469974824     sue  advanced.html                 31    NaN     NaN
1469974854     sue  advanced.html                 32    NaN     NaN
1469974924     sue  advanced.html                 33    NaN     NaN
1469977424     bob     intro.html                  1   True    10.0
1469977454     bob     intro.html                  1    NaN     NaN
1469977484     bob     intro.html                  1    NaN     NaN
1469977514     bob     intro.html                  1    NaN     NaN
1469977544     bob     intro.html                  1    NaN     NaN
1469977574     bob     intro.html                  1    NaN     NaN
1469977604     bob     intro.html                  1    NaN     NaN
1469977634     bob     intro.html                  1    NaN     NaN
1469977664     bob     intro.html                  1    NaN     NaN
1469977694     bob     intro.html                  1    NaN     NaN
1469977724     bob     intro.html                  1    NaN     NaN

In [128]:
df = df.reset_index()
df = df.set_index(['time', 'user'])
df

video  playback position paused  volume
time       user                                                   
1469974424 cheryl     intro.html                  5  False    10.0
           sue     advanced.html                 23  False    10.0
1469974454 cheryl     intro.html                  6    NaN     NaN
           sue     advanced.html                 24    NaN     NaN
1469974484 cheryl     intro.html                  7    NaN     NaN
1469974514 cheryl     intro.html                  8    NaN     NaN
1469974524 sue     advanced.html                 25    NaN     NaN
1469974544 cheryl     intro.html                  9    NaN     NaN
1469974554 sue     advanced.html                 26    NaN     NaN
1469974574 cheryl     intro.html                 10    NaN     NaN
1469974604 cheryl     intro.html                 11    NaN     NaN
1469974624 sue     advanced.html                 27    NaN     NaN
1469974634 cheryl     intro.html                 12    NaN     NaN
1469974654 sue     advanced.html                 28    NaN     5.0
1469974664 cheryl     intro.html                 13    NaN     NaN
1469974694 cheryl     intro.html                 14    NaN     NaN
1469974724 cheryl     intro.html                 15    NaN     NaN
           sue     advanced.html                 29    NaN     NaN
1469974754 sue     advanced.html                 30    NaN     NaN
1469974824 sue     advanced.html                 31    NaN     NaN
1469974854 sue     advanced.html                 32    NaN     NaN
1469974924 sue     advanced.html                 33    NaN     NaN
1469977424 bob        intro.html                  1   True    10.0
1469977454 bob        intro.html                  1    NaN     NaN
1469977484 bob        intro.html                  1    NaN     NaN
1469977514 bob        intro.html                  1    NaN     NaN
1469977544 bob        intro.html                  1    NaN     NaN
1469977574 bob        intro.html                  1    NaN     NaN
1469977604 bob        intro.html                  1    NaN     NaN
1469977634 bob        intro.html                  1    NaN     NaN
1469977664 bob        intro.html                  1    NaN     NaN
1469977694 bob        intro.html                  1    NaN     NaN
1469977724 bob        intro.html                  1    NaN     NaN

In [129]:
df = df.fillna(method = 'ffill')
df.head()

video  playback position  paused  volume
time       user                                                    
1469974424 cheryl     intro.html                  5   False    10.0
           sue     advanced.html                 23   False    10.0
1469974454 cheryl     intro.html                  6   False    10.0
           sue     advanced.html                 24   False    10.0
1469974484 cheryl     intro.html                  7   False    10.0